In [1]:
import numpy as np
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod

In [2]:
class Solver(ABC):
    """A solver. It may be initialized with some hyperparameters."""

    @abstractmethod
    def get_parameters(self):
        """Returns a dictionary of hyperparameters"""
        ...

    @abstractmethod
    def solve(self, problem, x0, *args, **kwargs):
        """
        A method that solves the given problem for given initial solution.
        It may accept or require additional parameters.
        Returns the solution and may return additional info.
        """



In [18]:
class GradientDescent(Solver):
    def __init__(self, beta, iterations):
        self.beta = beta
        self.iterations = iterations

    def get_parameters(self):
        dict = {}
        dict["Beta"] = self.beta
        dict["Iterations"] = self.iterations
        return dict
    
    def solve(self, problem, x0):
        while self.iterations > 0:
           self.iterations -= 1
           d = problem(x0)
           x0 = np.subtract(x0, d * self.beta)
        #    print(x0, d)
        return x0
    
    @staticmethod
    def gradient_f1(x):
        return x**3
    
    @staticmethod
    def gradient_f2(x0):
        x1, x2 = x0
        return np.array(2 * x1 * np.exp(-x1**2 - x2**2) + (x1-1) * np.exp(-(x1 - 1)**2 - (x2 - 1)**2),
                        2 * x2 * np.exp(-x1**2 - x2**2) + (x2-1) * np.exp(-(x1 - 1)**2 - (x2 - 1)**2))
    
    @staticmethod
    def f1(x):
        return 1/4 * x**4
    
    @staticmethod
    def f2(x):
        x1, x2 = x
        return 1.5 - np.exp(-x1**2 - x2**2) - 0.5 * np.exp(-(x1 - 1)**2 - (x2 + 2)**2)
    

In [19]:
a = GradientDescent(0.07, 10000)
a.solve(a.gradient_f2, (2.15, 3.19))
plotter(a.f2, title="$f(x) = \\frac{1}{4}x^4$", y_label='f(x)')

TypeError: cannot unpack non-iterable numpy.float64 object